In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from fuzzywuzzy import fuzz
import evaluate
import csv
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import top_k_accuracy_score, ndcg_score
import random

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def get_first_five_words(sentence):
    words = sentence.split()  # Split the sentence into a list of words
    return " ".join(words[:10])  # Join the first 5 words back into a string

In [4]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/amazon_electronics/splits/train.jsonl"
df_recommender_train = pd.read_json(input_file, lines=True)
for _, row in df_recommender_train.iterrows():
    row["recommended_product"]["product_name"] = row["recommended_product"]["product_name"].lower()

In [5]:
df_recommender_train

,user_id,user_previous_interactions,recommended_product,negative_recommended_product,turns
0,AFAOAYA6HR7HNJULFPHIZEDMT5TQ,[],{'product_name': 'acoustic research ht191 comp...,[{'product_name': 'JSAUX USB C to Lightning Ca...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,AFVDLVJBNUMQSFNLWNZG3TNKLKOA,[],{'product_name': 'kodak dc280 2mp digital came...,[{'product_name': 'Kodak PIXPRO Astro Zoom AZ2...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,AG6NTNXLUYALRUIHCSP3SSZBIFWQ,[],{'product_name': 'sony mvc-fd88 mavica 1.2mp d...,[{'product_name': 'Fujifilm XP70 16 MP Digital...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,AGIBU65Z2V6GHDA7OO4RSK62ABYA,[],{'product_name': 'samsung maxima 70gl power zo...,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,AEI27URBWEHBGVRODR74ZL4LPGQQ,[],"{'product_name': 'sony dvp-s550d dvd player', ...",[{'product_name': 'Sony DVP-FX820/W 8-Inch Por...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
8491,AGH4YIW3THVSFYUMVXMJC57W5G7A,[{'product_name': 'JETech Case Compatible with...,"{'product_name': 'powered usb hub, rshtech 4 p...",[{'product_name': 'Anker 4-Port USB 3.0 Unibod...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8492,AFWYLAZVW7SLPGC3JLHT6BQE34XA,[{'product_name': 'Cardo PTB00040 - PACKTALK S...,{'product_name': 'wealpe jaws clamp mount flex...,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8493,AHPNDZ62QLYS6OQRQ7UGZ4P36QQA,[{'product_name': 'ASURION 3 Year Home Improve...,{'product_name': 'mission darkness non-window ...,[{'product_name': 'MoKo Case for All-New Amazo...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8494,AFLP5GUTRZPAS37FV3F4LWM6ZNEQ,[{'product_name': 'Qoosea Compatible with Appl...,{'product_name': 'esr trifold case for ipad 9/...,[{'product_name': 'MoKo iPad 10.2 Case for iPa...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [6]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/amazon_electronics/splits/val.jsonl"
df_recommender_validation = pd.read_json(input_file, lines=True)
for _, row in df_recommender_validation.iterrows():
    row["recommended_product"]["product_name"] = row["recommended_product"]["product_name"].lower()

In [6]:
df_recommender_validation

,user_id,user_previous_interactions,recommended_product,negative_recommended_product,turns
0,AFILTCNQNPIQUMTPPEO7YAKDDLYA,[{'product_name': 'Corsair Dual SSD Mounting B...,{'product_name': 'bluetooth 5.0 wireless earbu...,[{'product_name': 'TOZO NC9 2022 Version Hybri...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,AFDLB72VMHV6MCFF2OQGYKAKZ37A,[{'product_name': 'ULANZI OP-4 OSMO Pocket WiF...,{'product_name': 'utebit camera bike mount cla...,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,AGFJIKKLBKCDI3PY3VO2VB2OLMJA,[{'product_name': 'tomtoc 13.5-14.4 Inch Lapto...,{'product_name': 'president electronics bill c...,[{'product_name': 'President Electronics MC Ki...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,AF5TH4CV76JKHHGPSQCDNZPPFZSQ,[{'product_name': 'GoPro Shorty Mini Extension...,{'product_name': 'asimom bluetooth speaker for...,[{'product_name': 'Bose SoundLink Color Blueto...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,AGOI5T2F2MC74LDESY2E2KVZXIEQ,[{'product_name': 'MSI GAMING GeForce GT 710 2...,{'product_name': 'visiontek radeon 5450 2gb dd...,[{'product_name': 'EVGA GeForce GTX 1070 FTW G...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
1815,AFJCRAOZ5NPZYWYA3D5J6HX7AAEQ,[{'product_name': 'Anker 7-Port USB 3.0 Data H...,{'product_name': 'ham radio handheld (uv-5r 8w...,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1816,AHEXBRMEKDPO3SHDKXHNYEYEKEHQ,[{'product_name': 'Amazon Basics Single Monito...,{'product_name': 'dbillionda cat8 ethernet cab...,"[{'product_name': 'Cat8 Ethernet Cable, Outdoo...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1817,AGMV44HAYRTPO7XUYT2JNMSJNZUQ,[{'product_name': 'Kindle (2019 release) - Now...,{'product_name': 'asus vivobook s15 s533 thin ...,[{'product_name': 'Acer Chromebook Spin 314 Co...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1818,AEDOHFM25ZBJNS3VNSOZ62IM2IQA,[{'product_name': 'Apple 87W USB-C Power Adapt...,"{'product_name': 'vemont usb c hub,5 in1 usb-c...",[{'product_name': 'Anker 4-Port USB 3.0 Unibod...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
prompt_train = []
recommend_train = []
not_founds = 0

for _, row in tqdm(df_recommender_train.iterrows(), total=len(df_recommender_train)):
    prompt = ""
    found = False
    recommended = get_first_five_words(row["recommended_product"]["product_name"])
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
        
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_train.append(prompt)
                recommend_train.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"len(prompt_train): {len(prompt_train)}")
print(f"len(recommend_train): {len(recommend_train)}")

            
            
prompt_encodings = tokenizer(prompt_train, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_train, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_train = Dataset.from_dict(dataset)

  0%|          | 0/8496 [00:00<?, ?it/s]

100%|██████████| 8496/8496 [01:41<00:00, 83.88it/s] 


Could not find 147
len(prompt_train): 8349
len(recommend_train): 8349


In [9]:
prompt_validation = []
recommend_validation = []
not_founds = 0

for _, row in tqdm(df_recommender_validation.iterrows(), total=len(df_recommender_validation)):
    prompt = ""
    found = False
    recommended = get_first_five_words(row["recommended_product"]["product_name"])
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
            
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_validation.append(prompt)
                recommend_validation.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1
        
print(f"Could not find {not_founds}")
print(f"len(prompt_validation): {len(prompt_validation)}")
print(f"len(recommend_validation): {len(recommend_validation)}")
            
            
prompt_encodings = tokenizer(prompt_validation, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_validation, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_validation = Dataset.from_dict(dataset)

  0%|          | 0/1820 [00:00<?, ?it/s]

100%|██████████| 1820/1820 [00:23<00:00, 77.16it/s]


Could not find 47
len(prompt_validation): 1773
len(recommend_validation): 1773


In [10]:
def data_collator(batch):
    input_ids, attention_mask, labels,  = [], [], []
    for sample in batch:
        input_ids.append(sample['input_ids'])
        attention_mask.append(sample['attention_mask'])
        labels.append(sample['labels'])
    max_encoder_len = max(sum(x) for x in attention_mask)
    max_decoder_len = max(sum([0 if item == IGNORE_INDEX else 1 for item in x]) for x in labels)
    return {
        'input_ids': torch.tensor(input_ids)[:, :max_encoder_len],
        'attention_mask': torch.tensor(attention_mask)[:, :max_encoder_len],
        'labels': torch.tensor(labels)[:, :max_decoder_len]
    }

In [11]:
training_args = TrainingArguments(
    output_dir="/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/amazon_electronics/T5_recommender",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=0.3,#self.cfg.save_steps,
    eval_steps=0.3, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=3, #gradient_accumulation_steps,
    per_device_train_batch_size=4, #train_batch_size,
    per_device_eval_batch_size=4, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_validation,
        data_collator=data_collator,
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
trainer.train() # resume_from_checkpoint=True
trainer.save_model()

Step,Training Loss,Validation Loss
1044,2.912800,2.780494
2088,2.662600,2.687867
3132,2.540900,2.665281


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


# Load the model and test it on the test dataset

In [4]:
torch.cuda.empty_cache()

In [5]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/amazon_electronics/splits/test.jsonl"
df_recommender_test = pd.read_json(input_file, lines=True)
for _, row in df_recommender_test.iterrows():
    row["recommended_product"]["product_name"] = row["recommended_product"]["product_name"].lower()

In [6]:
apps_training_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/amazon_electronics/electronics_df.csv"

all_apps = []
with open(apps_training_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        all_apps.append(get_first_five_words(row["title"].lower()))
        
all_apps = list(set(all_apps))

In [7]:
def candidate_creator(row):
    np.random.seed(row.name+30)
    selected_values = np.random.choice(np.setdiff1d( all_apps, [get_first_five_words(row["recommended_product"]["product_name"])]), 24, replace=False) # filter_candidate_apps(row["recommended_book"]["book_name"])
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, get_first_five_words(row["recommended_product"]["product_name"])) 

df_recommender_test['candidate'] = df_recommender_test.apply(lambda row: candidate_creator(row), axis=1)

In [8]:
prompt_test = []
recommend_test = []
candidate_books = []
true_candidate_indexes = []
not_founds = 0
for _, row in df_recommender_test.iterrows():
    candidates = []
    for index, candidate_book in enumerate(row["candidate"].tolist()):
        candidates.append(candidate_book)
        if candidate_book == get_first_five_words(row["recommended_product"]["product_name"]):
            true_candidate_index = index
    prompt = ""
    
    found = False
    recommended = get_first_five_words(row["recommended_product"]["product_name"])
    
    for index, turn in enumerate(row["turns"]):
        computer = turn["COMPUTER"]
        
        if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
            prompt += "computer: I would recommend the "
            prompt_test.append(prompt)
            recommend_test.append(recommended)
            candidate_books.append(candidates)
            true_candidate_indexes.append(true_candidate_index)
            found = True
            break
        else:
            prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"Number of prompt: {len(prompt_test)}")
print(f"Number of generations: {len(recommend_test)}")
print(f"Number of candidate apps: {len(candidate_books)}")
print(f"Number of true candidate indexes: {len(true_candidate_indexes)}")

Could not find 50
Number of prompt: 1772
Number of generations: 1772
Number of candidate apps: 1772
Number of true candidate indexes: 1772


In [9]:
candidate_books[40]

['netgear ac1200 wireless wifi range extender (ex6200)',
 'id-cooling iceflow 240 vga graphic card cooler 240mm water cooler',
 'dent wireless car charger and mount with infrared smart sensor',
 'logitech mobile freedom bluetooth headset',
 'videosecu mounts lcd led plasma tv tilting wall mount fits',
 'tp-link ac1900 smart wireless router - beamforming dual band gigabit',
 'tze-251 24mm label tape label kingdom replacement for brother p-touch',
 'apple ipad with wifi, 32gb, space gray (2017 model)',
 'anytone at-d878uv plus d878uv d868uv d858 battery pack 7.4v 3100mah',
 'esr protective cover for airpods pro (2019 release), metro light',
 'gintai dc jack cable kd4t9 0kd4t9 for dell inspiron 15-5000',
 'tcl mtro100bt wireless in-ear earbuds noise isolating bluetooth headphones with',
 'case logic black black zoom camera case',
 'western digital my book premium 500gb usb/fw 400 ext hard',
 'vinabty replacement remote control 1 year warranty compatible with roku',
 'sandisk extreme pro-s

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/amazon_electronics/T5_recommender")
model.eval()
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70):
  prompt_batches = list(chunk(prompt_test, batch_size))
  generation_batches = list(chunk(recommend_test, batch_size))

  correctly_predicted = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to('cuda'), attention_mask=inputs["attention_mask"].to('cuda'),
                            max_new_tokens=32,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    decoded_generations = [tokenizer.decode(generation, skip_special_tokens=True, clean_up_tokenization_spaces=True) for generation in generations_predicted]
    generation_batch = [generation for generation in generation_batch]
    
    correctly_predicted.extend([1 if fuzz.ratio(predicted, ground_truth) > threshold else 0 for predicted, ground_truth in zip(decoded_generations, generation_batch)])

  return correctly_predicted

In [11]:
correctly_predicted = evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70)
success_rate = sum(correctly_predicted) / len(correctly_predicted)
print("success_rate: ", success_rate)

100%|██████████| 222/222 [04:43<00:00,  1.28s/it]

success_rate:  0.07844243792325056


In [12]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]
    
def convert_to_sublists(numbers, sublist_size):
    return [numbers[i:i+sublist_size] for i in range(0, len(numbers), sublist_size)]

def recommender_rank(prompts, candidate_apps, model, tokenizer, batch_size=8):
  model.eval()
  encoder_max_length = 1024
  decoder_max_length = 32
  prompts_tokenized = tokenizer(prompts, max_length=encoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
  
  input_ids_decoder = []
  attention_mask_decoder = []
  input_ids_encoder = []
  attention_mask_encoder  = []
  for index, candidate_app_elements in enumerate(candidate_apps):
    candidate_app_elements = [tokenizer.pad_token+element for element in candidate_app_elements] # adding pad token to the beginning of each candidate app
    candidate_apps_tokenized = tokenizer(candidate_app_elements, max_length=decoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
    for candidate_app_index in range(len(candidate_app_elements)):
      input_ids_decoder.append(candidate_apps_tokenized["input_ids"][candidate_app_index])
      attention_mask_decoder.append(candidate_apps_tokenized["attention_mask"][candidate_app_index])
      input_ids_encoder.append(prompts_tokenized["input_ids"][index])
      attention_mask_encoder.append(prompts_tokenized["attention_mask"][index])
  
  input_ids_encoder_batches = list(chunk(input_ids_encoder, batch_size))
  attention_mask_encoder_batches = list(chunk(attention_mask_encoder, batch_size))
  input_ids_decoder_batches = list(chunk(input_ids_decoder, batch_size))
  attention_mask_decoder_batches = list(chunk(attention_mask_decoder, batch_size))
  

  scores = []
  for input_ids_encoder_batch, attention_mask_encoder_batch, input_ids_decoder_batch, attention_mask_decoder_batch in tqdm(zip(input_ids_encoder_batches, attention_mask_encoder_batches, input_ids_decoder_batches, attention_mask_decoder_batches), total = len(input_ids_encoder_batches)):
    decoder_input_ids = torch.stack(input_ids_decoder_batch).to("cuda")
    decoder_attention_mask = torch.stack(attention_mask_decoder_batch).to("cuda")
    input_ids = torch.stack(input_ids_encoder_batch).to("cuda")
    attention_mask = torch.stack(attention_mask_encoder_batch).to("cuda")
    with torch.no_grad():
      model_output = model(decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, 
                           input_ids=input_ids, attention_mask=attention_mask)
    
    logprobs = F.log_softmax(model_output["logits"], dim=-1)[:, :-1, :] # remove the eos token
    output_tokens = decoder_input_ids[:, 1:] # remove the bos token
        
    tokens_logprobs = torch.gather(logprobs, 2, output_tokens[:, :, None]).squeeze(-1).to(torch.float32)
        
    mask = torch.ones(tokens_logprobs.shape, dtype=torch.bool, device="cuda")
    for i, _output in enumerate(output_tokens):
      for j, _token in enumerate(_output):
        if _token == tokenizer.pad_token_id:
          mask[i, j] = False
              
    score = (tokens_logprobs * mask).sum(-1) / mask.sum(-1)
    scores.extend(score.to('cpu').tolist())
    
  # batch_input_representations = torch.cat(batch_input_representations)
  
  scores = convert_to_sublists(scores, len(candidate_apps[0]))
  
  return scores

In [13]:
scores = recommender_rank(prompt_test, candidate_books, model, tokenizer, batch_size=16)

100%|██████████| 2769/2769 [36:05<00:00,  1.28it/s]


Sampled Candidates

In [14]:
# that is the one
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

[np.float64(0.5524830699774267),
 np.float64(0.6811512415349887),
 np.float64(0.7528216704288939),
 np.float64(0.7996613995485328),
 np.float64(0.8329571106094809),
 np.float64(0.8639954853273137),
 np.float64(0.8860045146726863),
 np.float64(0.909706546275395),
 np.float64(0.9243792325056434),
 np.float64(0.9334085778781038)]

In [15]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [16]:
# that is the one
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]

[np.float64(0.5524830699774267),
 np.float64(0.6336636477507294),
 np.float64(0.669498862197682),
 np.float64(0.6896716355160181),
 np.float64(0.702552174808816),
 np.float64(0.71360826695946),
 np.float64(0.7209446100745841),
 np.float64(0.7284217685537041),
 np.float64(0.7328386872259747),
 np.float64(0.7354487533778292)]

Similar Candidates

In [ ]:
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

In [44]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [ ]:
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]